<a href="https://colab.research.google.com/github/omniaabo/Test/blob/main/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 30 20:05:56 2022

@author: TOSHIBA
"""
#initialize Switch_Controller_Mapping
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from keras.utils.np_utils import to_categorical
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np


print(tf.config.list_physical_devices('GPU'))
Switch_Controller = [(9,12),(6,13),(5,14)]
pathes={"ATLAM5":[0,1,5,14],"ATLAng":[1,5,14],"CHINng":[2,5,14],"DNVRng":[3,6,13],"HSTNng":[4,6,13],"IPLSng":[5,14],
         "KSCYng":[6,13],"LOSAng":[7,9,12],"NYCMng":[8,2,5,14],"SNVAng":[9,12],"STTLng":[10,9,12],"WASHng":[11,8,2,5,14]}
Mapping = pd.DataFrame.from_dict(pathes, orient='index').transpose()

class DQN(keras.Model):
    '''
    def __init__(self,sequences,discount_rate,Epsilon_start,Epsilon_end,Epsilon_decay):
        self.sequences = sequences
        self.discount_rate = discount_rate
        self.Epsilon_start = Epsilon_start
        self.Epsilon_end = Epsilon_end
        self.Epsilon_decay = Epsilon_decay
        
    '''
    def __init__(self,n_actions,fc1_dims,fc2_dims):
        super(DQN, self).__init__()
        self.dense1 = keras.layers.Dense(fc1_dims,activation='relu')
        self.dense2 = keras.layers.Dense(fc2_dims,activation='relu')
        self.V = keras.layers.Dense(1,activation=None)
        self.A = keras.layers.Dense(n_actions,activation=None)
        
    def call(self,state):
        x = self.dense1(state)
        x = self.dense2(x)
        A = self.A(x)
        return A
    
class ReplayBuffer():
    def __init__(self,max_size,input_shape):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size,*input_shape),dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size,*input_shape),dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size,dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size,dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size,dtype=np.bool)
        
        
    def store_transition(self,state,action,reward,state_,done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done
        self.mem_cntr +=1
        
    def sample_buffer(self,batch_size):
        max_mem = min(self.mem_cntr,self.mem_size)
        batch = np.random.choice(max_mem,batch_size,replace=False)
        states = self.state_memory[batch]
        new_states = self.new_state_memory 
        actions = self.action_memory[batch]
        reward = self.reward_memory[batch]
        dones = self.terminal_memory[batch]
        return states,actions,reward,new_states,dones

class Agent():
  def __init__(self,lr,gamma,n_actions,epsilon,batch_size,input_dims
               ,eps_dec=10_000,eps_end=0.02,mem_size=100_000,
               fname='dqn.h5',fc1_dims=128,fc2_dims=128,replace=100):
    self.action_sapce = [i for i in range(n_actions)]
    self.gamma = gamma
    self.epsilon = epsilon
    self.eps_dec = eps_dec
    self.eps_end = eps_end 
    self.fname = fname
    self.replace = replace
    self.learn_step_counter = 0
    self.memory = ReplayBuffer(mem_size,input_dims)
    self.q_eval = DQN(n_actions,fc1_dims,fc2_dims)
    self.q_new = DQN(n_actions,fc1_dims,fc2_dims)
    self.q_eval.compile(optimizers=Adam(learning_rate=lr),loss='mean_squared_error')
    self.q_new.compile(optimizers=Adam(learning_rate=lr),loss='mean_squared_error')

  def store_transition(self,state,action,reward,new_state,done):
    self.memory.store_transition(state,action,reward,new_state,done)

  def choose_action(self,obseration):
    if np.random.random() < self.epsilon:
      action = np.random.choice(self.action_space)
    else:
      state = np.array([obseration])
      actions = self.q_eval.advantage(state)
      action = tf.math.argmax(actions,axis=1).numpy()[0]
    return action

  def learn(self):
    if self.mem_cntr < self.batch_size:
      return
    if self.learn_step_counter % self.replace == 0:
      self.q_next.set_weights(self.q_eval.get_weights())
    states,actions,rewards,states_,dones = self.memory.sample_buffer(self.batch_size)

    q_pred = self.q_eval(states)
    q_next = tf.max.reduce_max(self.q_next(states_),axis=1,keepdims=True).numpy()
    q_target = np.copy(q_pred)
    




[]
